In [1]:
import pandas as pd
import numpy as np

In [2]:
games = pd.read_csv('../data/appstore_games.csv')

In [3]:
original_colums_dict = {x: x.lower().replace(' ','_') for x in games.columns}

games.rename(
    columns = original_colums_dict,
    inplace = True
)

games.set_index(
    keys = 'id',
    inplace = True
)

games.drop(
    columns = ['url', 'icon_url'],
    inplace = True
)

games['original_release_date'] = pd.to_datetime(games['original_release_date'])
games['current_version_release_date'] = pd.to_datetime(games['current_version_release_date'])

games = games.loc[games['average_user_rating'].notnull()]

games = games.loc[games['user_rating_count'] >= 30]

In [4]:
games.shape

(4311, 15)

In [5]:
games['languages'] = games['languages'].fillna('EN')

In [6]:
games['free_game'] = (games['price'] == 0).astype(int).map({0:'paid', 1:'free'})

In [7]:
number_of_languages = games['languages'].str.split(',').apply(lambda x: len(x))
games['multilingual'] = number_of_languages == 1
games['multilingual'] = games['multilingual'].astype(int).map({0:'multilingual', 1:'monolingual'})

In [8]:
games['price_language'] = games['free_game'] + '-' + games['multilingual']

In [9]:
games['price_language'].value_counts()

free-monolingual     2105
free-multilingual    1439
paid-monolingual      467
paid-multilingual     300
Name: price_language, dtype: int64

In [10]:
games.groupby('price_language')['average_user_rating'].describe()

,count,mean,std,min,25%,50%,75%,max
price_language,,,,,,,,
free-monolingual,2105.0,4.129216,0.634426,1.5,4.0,4.5,4.5,5.0
free-multilingual,1439.0,4.227589,0.525904,1.5,4.0,4.5,4.5,5.0
paid-monolingual,467.0,4.064240,0.657693,1.5,3.5,4.0,4.5,5.0
paid-multilingual,300.0,4.251667,0.486435,2.5,4.0,4.5,4.5,5.0
